In [1]:
!git clone https://github.com/Oliverkkkk/CITS4403.git
%cd CITS4403

fatal: destination path 'CITS4403' already exists and is not an empty directory.
/content/CITS4403


In [2]:
%cd /content/CITS4403
!pip install -r requirements.txt


/content/CITS4403


In [3]:
import numpy as np, os
os.makedirs("/content/CITS4403/data", exist_ok=True)

H, W = 25, 25


veg = np.random.choice([0,1,2,3,4], size=(H, W),
                       p=[0.4,0.2,0.15,0.15,0.1]).astype(np.int16)
np.savetxt("/content/CITS4403/data/veg.csv", veg, fmt="%d", delimiter=",")


river = np.zeros((H, W), dtype=bool)
thickness = 2
cx = W // 2
for y in range(H):
    rx = int(cx + 2*np.sin(2*np.pi*y/max(1,H)))
    xL, xR = max(0, rx - thickness//2), min(W, rx + (thickness+1)//2)
    river[y, xL:xR] = True
np.savetxt("/content/CITS4403/data/river.csv", river.astype(int), fmt="%d", delimiter=",")

print("Saved:", "/content/CITS4403/data/veg.csv", "/content/CITS4403/data/river.csv")


Saved: /content/CITS4403/data/veg.csv /content/CITS4403/data/river.csv


In [4]:
import numpy as np
from PIL import Image
import os


def load_veg_csv(path):   # (H,W) -> (W,H)
    v = np.loadtxt(path, delimiter=",", dtype=int)
    v = np.clip(v, 0, 4)
    return v.T.astype(np.int16)

def load_veg_png(path):   # (H,W) -> (W,H)
    a = np.array(Image.open(path).convert("L"), dtype=np.float32) / 255.0
    v = np.rint(a * 4).astype(np.int16)
    return np.clip(v, 0, 4).T

def load_river_csv(path): # (H,W) -> (W,H)
    m = np.loadtxt(path, delimiter=",", dtype=float)
    return (m > 0.5).T

def load_river_png(path): # (H,W) -> (W,H)
    a = np.array(Image.open(path).convert("L"))
    return (a >= 128).T

veg_path   = "/content/CITS4403/data/veg.csv"
river_path = "/content/CITS4403/data/river.csv"

V = load_veg_csv(veg_path)   if veg_path.endswith(".csv")   else load_veg_png(veg_path)
R = load_river_csv(river_path) if river_path.endswith(".csv") else load_river_png(river_path)

V.shape, R.shape, V.min(), V.max(), R.sum()


((25, 25), (25, 25), np.int16(0), np.int16(4), np.int64(50))

In [7]:
import numpy as np
from src.model import FeralCatModel
from src.visual2d import animate_grid
import matplotlib.pyplot as plt
from matplotlib import animation


V = np.loadtxt("/content/CITS4403/data/veg.csv", delimiter=",", dtype=int).T
R = np.loadtxt("/content/CITS4403/data/river.csv", delimiter=",", dtype=int).T > 0

model = FeralCatModel(width=V.shape[0], height=V.shape[1],
                      n_cats=6, n_prey=40,
                      predation_base=0.2, predation_coef=0.1,
                      prey_flee_prob=0.4,
                      vegetation=V, river=R)

model.datacollector.collect(model)
fig, anim = animate_grid(model, steps=100, interval_ms=150)
anim.save("/content/sim.gif", writer=animation.PillowWriter(fps=8))


In [10]:
from IPython.display import Video
display(Video("/content/sim.mp4", embed=True))
